# Detecting American Sign Language using K-Nearest Neighbors method


In [20]:
# Imports for this project

import os 
from PIL import Image
import numpy as np
import cv2
from sklearn.neighbors import KNeighborsClassifier

In [3]:
def load_training_dataset(directory):
    """Loads ASL training images

    Args:
        directory (str): directory path

    Returns:
        ndarray: training images and their labels
    """
    images = []
    labels = []

    for label in sorted(os.listdir(directory)):  # Ensure consistent label ordering
        subdirectory = os.path.join(directory, label)
        if os.path.isdir(subdirectory):  # Only process directories
            for filename in os.listdir(subdirectory):
                if filename.endswith('.jpg'):
                    filepath = os.path.join(subdirectory, filename)
                    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
                    if image is not None:
            # Resize image to ensure uniformity
            image = cv2.resize(image, (64, 64))
            # Apply Canny edge detection
            edges = cv2.Canny(image, 100, 200)  # Adjust thresholds as needed
            # Normalize pixel values to 0-1 range
            edges_normalized = edges / 255.0
            images.append(edges_normalized.flatten())  # Flatten the matrix to a vector
            labels.append(label)
    
    return np.array(images), np.array(labels)

def load_testing_images(directory):
    """Loads ASL testing images

    Args:
        directory (str): directory path

    Returns:
        ndarry: testing images and their labels
    """
    images = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith("_test.jpg"):  
            # Extract the label from the filename (assuming format 'A_test.jpg', 'B_test.jpg', etc.)
            label = filename.split('_')[0]
            
            # Construct the full path to the file
            file_path = os.path.join(directory, filename)
            
            # Open and process the image
            image = Image.open(file_path).convert('L')  # Convert image to grayscale
            image = image.resize((64, 64))  # Resize the image to 64x64 pixels
            
            # Convert image data to a numpy array and flatten it
            images.append(np.array(image).flatten())
            labels.append(label)
    
    # Convert lists to numpy arrays for easier handling in machine learning models
    return np.array(images), np.array(labels)

training_directory = "archive/asl_alphabet_train"
testing_directory = "archive/asl_alphabet_test"

training_images, training_labels = load_training_dataset(training_directory)
testing_images, testing_labels = load_testing_images(testing_directory)

In [4]:
print(f'Training dataset shape: {training_images.shape}')
print(f'Testing dataset shape: {testing_images.shape}')

Training dataset shape: (87000, 4096)
Testing dataset shape: (28, 4096)


In [ ]:
# Edge detection for images


def canny_edge(img):
	gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	blur_img = cv2.GaussianBlur(gray_img, (3,3), 0)
	
	canny_img = cv2.Canny(image=blur_img, threshold1=100, threshold2=200)
	
	return canny_img



In [5]:
# Creating a KNN model with the nearest 5 neighbors

knn_5 = KNeighborsClassifier(n_neighbors=5)
knn_5.fit(training_images,training_labels)

KNeighborsClassifier()

In [6]:
# Accuracy for when neighbors = 5

accuracy = knn_5.score(testing_images, testing_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 100.00%


In [7]:
# Creating a KNN model with the nearest 10 neighbors

knn_10 = KNeighborsClassifier(n_neighbors=10)
knn_10.fit(training_images,training_labels)


KNeighborsClassifier(n_neighbors=10)

In [8]:
# Accuracy for when neighbors = 10

accuracy = knn_10.score(testing_images, testing_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 96.43%


In [9]:
# Creating a KNN model with the nearest 15 neighbors

knn_15 = KNeighborsClassifier(n_neighbors=15)
knn_15.fit(training_images,training_labels)

KNeighborsClassifier(n_neighbors=15)

In [10]:
# Accuracy for when neighbors = 15

accuracy = knn_15.score(testing_images, testing_labels)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 96.43%


In [19]:
def predict_image(image_path, model):
    image = Image.open(image_path).convert('L')
    image = image.resize((64, 64))
    feature = np.array(image).flatten().reshape(1, -1)
    prediction = model.predict(feature)
    return prediction

l_test = "bjorn_images/IMG_3852.jpg"

predicted_label = predict_image(l_test,knn_15)

print(f"Predicted Label: {predicted_label}")


Predicted Label: ['nothing']
